In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
rating_df = pd.read_csv("C:/Users/Kalpesh__K__/Datasets/ratings.csv")

In [4]:
rating_df.head(10)

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523
5,1,110,4.0,1217896150
6,1,150,3.0,1217895940
7,1,161,4.0,1217897864
8,1,165,3.0,1217897135
9,1,204,0.5,1217895786


In [5]:
rating_df.describe

<bound method NDFrame.describe of         userId  movieId  rating   timestamp
0            1       16     4.0  1217897793
1            1       24     1.5  1217895807
2            1       32     4.0  1217896246
3            1       47     4.0  1217896556
4            1       50     4.0  1217896523
...        ...      ...     ...         ...
105334     668   142488     4.0  1451535844
105335     668   142507     3.5  1451535889
105336     668   143385     4.0  1446388585
105337     668   144976     2.5  1448656898
105338     668   148626     4.5  1451148148

[105339 rows x 4 columns]>

In [6]:
rating_df.shape

(105339, 4)

In [7]:
rating_df.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [8]:
movie_df = pd.read_csv("C:/Users/Kalpesh__K__/Datasets/movies.csv")

In [9]:
movie_df.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [10]:
movie_df.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [11]:
movie_df.shape

(10329, 3)

In [12]:
rating_df.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [13]:
movie_df.describe()

,movieId
count,10329.000000
mean,31924.282893
std,37734.741149
min,1.000000
25%,3240.000000
50%,7088.000000
75%,59900.000000
max,149532.000000


In [14]:
# Number of unique users in the ratings dataframe
num_users = rating_df['userId'].nunique()
print('Number of unique users:', num_users)

# Number of unique movies in the ratings dataframe
num_movies = rating_df['movieId'].nunique()
print('Number of unique movies:', num_movies)


Number of unique users: 668
Number of unique movies: 10325


In [15]:
# Extract all unique genres from the movies dataframe
unique_genres = set()
for genre_string in movie_df['genres']:
    genre_list = genre_string.split('|')
    unique_genres.update(genre_list)
    
print('Number of unique genres:', len(unique_genres))
print('Unique genres:', unique_genres)


Number of unique genres: 20
Unique genres: {'Western', 'Adventure', 'Animation', 'Mystery', 'Thriller', 'Action', 'Comedy', 'Crime', 'Film-Noir', '(no genres listed)', 'Children', 'Horror', 'War', 'IMAX', 'Drama', 'Romance', 'Musical', 'Fantasy', 'Sci-Fi', 'Documentary'}


In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data


# Compute the weighted rating of each movie
C = rating_df['rating'].mean()
m = rating_df.groupby('movieId').size()
movie_df['weighted_rating'] = (rating_df.groupby('movieId')['rating'].mean() * m) / (m + 1000) + (C * 1000) / (m + 1000)

# Create a TF-IDF matrix of movie genres
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movie_df['genres'].fillna(''))

# Define a function to get similar movies based on genres
def get_similar_movies(title, n_recommendations=5):
    # Get the index of the movie with the given title
    idx = movie_df[movie_df['title'] == title].index[0]

    # Compute the cosine similarity between the given movie and all other movies based on genres
    cosine_similarities = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)

    # Get the top N movies with the highest cosine similarity and highest weighted rating
    similar_movies = []
    for i in cosine_similarities.argsort()[0][::-1]:
        if i != idx:
            movie = movies.iloc[i]
            if len(similar_movies) >= n_recommendations:
                break
            if movie['weighted_rating'] >= 3.0:
                similar_movies.append(movie['title'])
    return similar_movies

# Test the function with an example
recommendations = get_similar_movies('Toy Story', n_recommendations=5)
print(recommendations)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [17]:
movie_df.columns

Index(['movieId', 'title', 'genres', 'weighted_rating'], dtype='object')

In [18]:
movie_df.columns

Index(['movieId', 'title', 'genres', 'weighted_rating'], dtype='object')

In [19]:
import pandas as pd

# Define the input parameters
genre = 'Action'
min_rating = 3
top_n = 5

# Filter the movies by the input genre and minimum rating threshold
movies = movie_df[movie_df['genres'].str.contains(genre)]
ratings = rating_df[rating_df['movieId'].isin(movies['movieId'])]
ratings = rating_df[rating_df['rating'] >= min_rating]





# Select the top N movies and return their titles and average ratings
top_movies = recommendations.head(top_n)
print(top_movies[['title', 'avg_rating']])


NameError: name 'recommendations' is not defined

In [21]:
# Calculate the popularity of each movie
popularity = ratings.groupby('movieId')['rating'].count().reset_index()
popularity.columns = ['movieId', 'popularity']


In [22]:
# Filter the movies by the popularity
min_popularity = 10
popular_movies = popularity[popularity['popularity'] >= min_popularity]
ratings = ratings[ratings['movieId'].isin(popular_movies['movieId'])]
print(popular_movies)
#print(top_movies[['title', 'avg_rating']])


      movieId  popularity
0           1         221
1           2          79
2           3          46
4           5          51
5           6         110
...       ...         ...
8599   119145          12
8610   122882          20
8612   122892          12
8674   134130          13
8680   134853          16

[1929 rows x 2 columns]


In [23]:
# Calculate the average rating for each movie
avg_ratings = ratings.groupby('movieId')['rating'].mean().reset_index()
avg_ratings.columns = ['movieId', 'avg_rating']
print(avg_ratings)

      movieId  avg_rating
0           1    4.004525
1           2    3.556962
2           3    3.510870
3           5    3.509804
4           6    4.163636
...       ...         ...
1924   119145    3.833333
1925   122882    3.950000
1926   122892    3.791667
1927   134130    3.961538
1928   134853    4.125000

[1929 rows x 2 columns]


In [24]:
# Merge the movie titles and average ratings
recommendations = pd.merge(movies[['movieId', 'title']], avg_ratings, on='movieId')
print(recommendations)

     movieId                                title  avg_rating
0          6                          Heat (1995)    4.163636
1          9                  Sudden Death (1995)    3.277778
2         10                     GoldenEye (1995)    3.698413
3         15              Cutthroat Island (1995)    3.300000
4         20                   Money Train (1995)    3.285714
..       ...                                  ...         ...
521   115617                    Big Hero 6 (2014)    4.400000
522   119145  Kingsman: The Secret Service (2015)    3.833333
523   122882            Mad Max: Fury Road (2015)    3.950000
524   122892       Avengers: Age of Ultron (2015)    3.791667
525   134130                  Martian, The (2015)    3.961538

[526 rows x 3 columns]


In [25]:
# Sort the movies by their average rating
recommendations = recommendations.sort_values('avg_rating', ascending=False)


print(recommendations)

     movieId                                     title  avg_rating
165     1927     All Quiet on the Western Front (1930)    4.500000
521   115617                         Big Hero 6 (2014)    4.400000
236     2959                         Fight Club (1999)    4.399471
208     2571                        Matrix, The (1999)    4.395161
242     3000  Princess Mononoke (Mononoke-hime) (1997)    4.384615
..       ...                                       ...         ...
4         20                        Money Train (1995)    3.285714
1          9                       Sudden Death (1995)    3.277778
315     4643                 Planet of the Apes (2001)    3.272727
210     2616                         Dick Tracy (1990)    3.138889
137     1499                           Anaconda (1997)    3.136364

[526 rows x 3 columns]


In [26]:
# Select the top N movies and return their titles and average ratings
top_movies = recommendations.head(top_n)
print(top_movies[['title', 'avg_rating']])


                                        title  avg_rating
165     All Quiet on the Western Front (1930)    4.500000
521                         Big Hero 6 (2014)    4.400000
236                         Fight Club (1999)    4.399471
208                        Matrix, The (1999)    4.395161
242  Princess Mononoke (Mononoke-hime) (1997)    4.384615


In [27]:


# Without specifying suffix
df = pd.concat([movie_df, rating_df], axis=1)
df = pd.concat([movie_df, rating_df.add_suffix('_2')], axis=1)


In [28]:
df.head(10)

,movieId,title,genres,weighted_rating,userId_2,movieId_2,rating_2,timestamp_2
0,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,1,16,4.0,1217897793
1,2.0,Jumanji (1995),Adventure|Children|Fantasy,3.590382,1,24,1.5,1217895807
2,3.0,Grumpier Old Men (1995),Comedy|Romance,3.503068,1,32,4.0,1217896246
3,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,3.498913,1,47,4.0,1217896556
4,5.0,Father of the Bride Part II (1995),Comedy,3.509249,1,50,4.0,1217896523
5,6.0,Heat (1995),Action|Crime|Thriller,3.501272,1,110,4.0,1217896150
6,7.0,Sabrina (1995),Comedy|Romance,3.574305,1,150,3.0,1217895940
7,8.0,Tom and Huck (1995),Adventure|Children,3.509811,1,161,4.0,1217897864
8,9.0,Sudden Death (1995),Action,3.517298,1,165,3.0,1217897135
9,10.0,GoldenEye (1995),Action|Adventure|Thriller,3.502298,1,204,0.5,1217895786


In [29]:
movie_df.columns


Index(['movieId', 'title', 'genres', 'weighted_rating'], dtype='object')

In [30]:
rating_df.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [31]:
import pandas as pd

# Load the movies and ratings datasets
#movies_df = pd.read_csv("movies.csv")
#ratings_df = pd.read_csv("ratings.csv")

#def popularity_recommender(genre, min_threshold, num_recommendations):
    # Filter the movies by the input genre
 #   genre_movies = movie_df[movie_df["genre"].str.contains(genre)]

    # Filter the ratings to only include movies with at least the min_threshold number of ratings
  #  ratings_count = rating_df["movieId"].value_counts()
   # ratings_filtered = rating_df[rating_df["movieId"].isin(ratings_count.index[ratings_count >= min_threshold])]

    # Merge the filtered movies and ratings
    #merged_df = pd.merge(ratings_filtered, genre_movies, on="movieId")

    # Calculate the popularity score for each movie
    #popularity_scores = merged_df.groupby(["movieId", "title"]).size().reset_index(name="popularity_score")

    # Sort the movies by the popularity score and rating
#    sorted_movies = popularity_scores.sort_values(["popularity_score", "rating"], ascending=False)

    # Return the top N movies
 #   top_movies = sorted_movies.head(num_recommendations)

  #  return top_movies[["title", "popularity_score"]]


In [32]:
#popularity_recommender("Comedy", 100, 5)







In [36]:
# User inputs
genre = input("Enter a genre: ")
min_rating_threshold = float(input("Enter a minimum rating threshold: "))
num_recommendations = int(input("Enter the number of recommendations: "))

# Filter the movies dataset by genre and minimum rating threshold
genre_movies = movie_df[movie_df['genres'].str.contains(genre)]
genre_ratings = ratings[ratings['movieId'].isin(genre_movies['movieId'].tolist())]
genre_ratings_counts = genre_ratings.groupby('movieId')['rating'].count().reset_index()
genre_ratings_counts.rename(columns={'rating':'num_ratings'}, inplace=True)
popular_movies = genre_ratings_counts[genre_ratings_counts['num_ratings'] >= min_rating_threshold]
popular_movies = popular_movies.merge(movies, on='movieId', how='left')

# Sort the popular movies by average rating and number of reviews
popular_movies['average_rating'] = popular_movies['movieId'].apply(lambda x: ratings[ratings['movieId']==x]['rating'].mean())
popular_movies = popular_movies.sort_values(['average_rating', 'num_ratings'], ascending=False)

# Display top N movie recommendations
print("Top {} recommendations for genre '{}':".format(num_recommendations, genre))
for i, row in popular_movies.head(num_recommendations).iterrows():
    print("{}. {} (Avg. rating: {:.2f}, Num. reviews: {})".format(i+1, row['title'], row['average_rating'], row['num_ratings']))


Enter a genre: Comedy
Enter a minimum rating threshold: 100
Enter the number of recommendations: 5
Top 5 recommendations for genre 'Comedy':
19. nan (Avg. rating: 4.43, Num. reviews: 146)
15. nan (Avg. rating: 4.33, Num. reviews: 196)
4. nan (Avg. rating: 4.32, Num. reviews: 305)
6. nan (Avg. rating: 4.30, Num. reviews: 291)
20. Princess Bride, The (1987) (Avg. rating: 4.28, Num. reviews: 162)


In [66]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie metadata from a CSV file
movies = pd.read_csv('C:/Users/Kalpesh__K__/Datasets/movies.csv')

# Specify the input parameters
movie_title = 'Toy Story'
num_recommendations = 5

# Calculate the similarity between movies based on genre
tfidf = TfidfVectorizer()
genre_matrix = tfidf.fit_transform(movies['genres'])
cos_sim = cosine_similarity(genre_matrix)

# Get the index of the input movie
#movie_index = movies[movies['title'] == movie_title].index[0]
input_title = 'Toy Story'
input_movie_row = movies[movies['title'] == input_title].iloc[0]
input_movie_index = input_movie_row.name


# Get the similarity scores between the input movie and all other movies
similarity_scores = cos_sim[movie_index]

# Sort the movies by similarity
similar_movies = list(enumerate(similarity_scores))
sorted_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

# Recommend the top N movies
recommendations = [movies.iloc[i[0]]['title'] for i in sorted_movies[1:num_recommendations+1]]

print('Top {} movies similar to {}:'.format(num_recommendations, movie_title))
for i, title in enumerate(recommendations):
    print('{}. {}'.format(i+1, title))


IndexError: single positional indexer is out-of-bounds

In [67]:

# Step 1: Calculate the similarity scores between the target user and all other users based on their ratings

# create a pivot table with users as rows, movies as columns and ratings as values
pivot_table = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# calculate the cosine similarity between the target user and all other users
from sklearn.metrics.pairwise import cosine_similarity
user_similarities = cosine_similarity(pivot_table.fillna(0))

# Step 2: Select the top K most similar users to the target user
k = 100  # threshold for similar users
target_user = 1  # user for whom recommendations are being made

# get the indices of the top k similar users to the target user
similar_users_indices = user_similarities[target_user-1].argsort()[-k-1:-1][::-1]

# Step 3: Identify the movies that the target user has not yet seen but have been seen and rated by the top K similar users
# get the movies that the target user has not yet seen
movies_not_seen = pivot_table.loc[target_user][pivot_table.loc[target_user].isnull()].index

# create a dataframe containing the ratings for these movies by the top k similar users
similar_users_ratings = pivot_table.loc[similar_users_indices, movies_not_seen]

# Step 4: Calculate the predicted ratings for the target user for these movies
# calculate the weighted average of the ratings for each movie by the top k similar users
predicted_ratings = similar_users_ratings.apply(lambda x: np.dot(x.fillna(0), user_similarities[target_user-1][similar_users_indices]) / user_similarities[target_user-1][similar_users_indices].sum(), axis=0)

# Step 5: Recommend the top N movies based on the predicted ratings
n = 5  # number of recommendations to make
top_n_movies = predicted_ratings.sort_values(ascending=False)[:n].index.tolist()

# print the top N recommended movies
print("Top {} recommended movies for user {}:".format(n, target_user))
for movie_id in top_n_movies:
    movie_title = movies[movies['movieId'] == movie_id]['title'].values[0]
    print("- " + movie_title)



Top 5 recommended movies for user 1:
- Speed (1994)
- Die Hard (1988)
- Toy Story (1995)
- Princess Bride, The (1987)
- Indiana Jones and the Last Crusade (1989)


##### 




In [68]:
import pandas as pd
import numpy as np

# Load data
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Merge data
movie_ratings = pd.merge(movies, ratings, on='movieId')

# Pivot table to get matrix of users and their movie ratings
user_movie_ratings = pd.pivot_table(movie_ratings, index='userId', columns='title', values='rating')

# Function to get top N recommended movies for target user
def get_top_n_recommended_movies(target_user_id, N, k):
    # Get target user's movie ratings
    target_user_ratings = user_movie_ratings.loc[target_user_id].dropna()
    
    # Find similar users based on cosine similarity
    cosine_similarities = user_movie_ratings.corrwith(target_user_ratings)
    similar_users = cosine_similarities.dropna().sort_values(ascending=False)[:k]
    
    # Get movie ratings of similar users
    similar_users_ratings = user_movie_ratings.loc[similar_users.index]
    
    # Calculate weighted average of movie ratings of similar users
    similar_users_weighted_ratings = (similar_users_ratings.T * similar_users).T.sum().div(similar_users.sum())
    
    # Filter out movies that the target user has already rated
    recommended_movies = similar_users_weighted_ratings.drop(target_user_ratings.index).sort_values(ascending=False)[:N]
    
    # Get movie titles for recommended movies
    recommended_movie_titles = pd.DataFrame(recommended_movies).merge(movies, left_index=True, right_on='title')['title']
    
    return recommended_movie_titles

# Example usage
target_user_id = int(input('Enter user ID: '))
N = int(input('Enter number of recommendations: '))
k = int(input('Enter threshold for similar users: '))

recommended_movies = get_top_n_recommended_movies(target_user_id, N, k)

print('Top', N, 'recommended movies for user', target_user_id, ':')
for i, movie_title in enumerate(recommended_movies):
    print(i+1, movie_title)


FileNotFoundError: [Errno 2] No such file or directory: 'movies.csv'